## Часть 1
Напишите скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227). Затем нужно перейти на сайты региональных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название региона
- название ТИК
- номер УИК
- 20 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

### Часть 2
Используя скаченные данные и `Pandas` (или `PySpark`) посчитайте:
- явку (%) по всем регионам, результат отсортировать по убыванию
- выберите произвольного кандидата и найдите тот избиратльный участок, на котором он получил наибольший результат  (%, учитывать участки на которых проголосовало больше 300 человек)
- найдите регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна
- посчитайте дисперсию по явке для каждого региона (учитывать УИК)
- для каждого кандидата посчитайте таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [65]:
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv('results.csv', index_col=0)

In [26]:
df['явка_количество_человек'] = df['Число недействительных избирательных бюллетеней'] \
                                + df['Число действительных избирательных бюллетеней']
df['явка_проценты'] = df['явка_количество_человек'] / df['Число избирателей, включенных в список избирателей ']

In [29]:
#посчитайте явку (%) по всем регионам, результат отсортировать по убыванию
temp = df.groupby('название региона').sum()
results = (temp['явка_количество_человек'] 
           / temp['Число избирателей, включенных в список избирателей ']).sort_values(ascending=False)
with pd.option_context('display.max_rows', 100):
    print(results)

название региона
Республика Тыва                             0.936254
Ямало-Ненецкий автономный округ             0.918721
Кабардино-Балкарская Республика             0.916979
Чеченская Республика                        0.915093
Республика Северная Осетия - Алания         0.899370
Республика Дагестан                         0.874435
Карачаево-Черкесская Республика             0.873438
Кемеровская область                         0.830756
Чукотский автономный округ                  0.822605
Республика Ингушетия                        0.819594
Брянская область                            0.796598
Тюменская область                           0.788830
Республика Мордовия                         0.778365
Краснодарский край                          0.778347
Республика Татарстан (Татарстан)            0.773653
Чувашская Республика - Чувашия              0.762106
Республика Башкортостан                     0.754212
Республика Бурятия                          0.751913
Республика Адыгея (Адыгея)   

In [59]:
#выберите произвольного кандидата и найдите тот избиратльный участок, 
#на котором он получил наибольший результат (%, учитывать участки на которых проголосовало больше 300 человек)

df.assign(процент = df['Грудинин Павел Николаевич'] / df['явка_количество_человек']).sort_values('процент', ascending=False)[
    (df['явка_количество_человек'] > 300)][['номер УИК', 'процент']].iloc[0]

c:\users\peksha\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


номер УИК    УИК №1306
процент       0.666472
Name: 55360, dtype: object

In [57]:
#найдите регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

temp = df.groupby(['название региона', 'название ТИК']).sum()

(temp['явка_количество_человек'] 
 / temp['Число избирателей, включенных в список избирателей ']).groupby('название региона').apply(
    lambda x: x.max() - x.min()).idxmax()

'Архангельская область'

In [63]:
#посчитайте дисперсию по явке для каждого региона (учитывать УИК)

with pd.option_context('display.max_rows', 100):
    print(df.groupby('название региона')['явка_проценты'].var())

название региона
Алтайский край                              0.009840
Амурская область                            0.013917
Архангельская область                       0.019866
Астраханская область                        0.012645
Белгородская область                        0.018929
Брянская область                            0.011135
Владимирская область                        0.005686
Волгоградская область                       0.014780
Вологодская область                         0.008314
Воронежская область                         0.022427
Еврейская автономная область                0.011762
Забайкальский край                          0.014354
Ивановская область                          0.010275
Иркутская область                           0.011342
Кабардино-Балкарская Республика             0.001385
Калининградская область                     0.014347
Калужская область                           0.013065
Камчатский край                             0.028471
Карачаево-Черкесская Республи

In [91]:
#для каждого кандидата посчитайте таблицу: результат (%, округленный до целого) - количество УИК, 
#на которых кандидат получил данный результат

percentage = np.arange(101)
names = df.columns[12:20]
res = np.zeros((len(names), 101))
for index, row in df.iterrows():
    c = row['явка_количество_человек']
    if c == 0:
        c = 1
    for i, name in enumerate(names):
        t = 100 * row[name] / c
        res[i, int(np.round(t))] += 1

In [94]:
pd.set_option('display.max_columns', None)
pd.DataFrame(res, columns=percentage, index=names)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
Бабурин Сергей Николаевич,44964.0,47709.0,3718.0,524.0,178.0,89.0,35.0,20.0,21.0,10.0,7.0,5.0,1.0,1.0,3.0,1.0,0.0,1.0,2.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Грудинин Павел Николаевич,1169.0,2138.0,3070.0,3356.0,3632.0,3830.0,3972.0,4502.0,5246.0,6057.0,7038.0,7716.0,8113.0,7188.0,6119.0,4846.0,3712.0,2921.0,2352.0,1883.0,1517.0,1281.0,1068.0,873.0,765.0,572.0,441.0,357.0,299.0,216.0,170.0,129.0,107.0,111.0,70.0,68.0,58.0,43.0,51.0,26.0,31.0,21.0,23.0,16.0,15.0,15.0,15.0,10.0,10.0,4.0,14.0,2.0,5.0,3.0,5.0,1.0,4.0,5.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Жириновский Владимир Вольфович,3981.0,4592.0,4737.0,6616.0,10247.0,13259.0,13730.0,11795.0,8847.0,6244.0,4060.0,2907.0,1922.0,1187.0,898.0,594.0,392.0,282.0,220.0,155.0,99.0,95.0,76.0,62.0,52.0,54.0,26.0,24.0,14.0,17.0,14.0,13.0,7.0,15.0,1.0,6.0,10.0,3.0,7.0,1.0,7.0,5.0,5.0,0.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Путин Владимир Владимирович,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,2.0,2.0,2.0,3.0,0.0,3.0,2.0,0.0,1.0,4.0,3.0,1.0,4.0,4.0,3.0,1.0,3.0,2.0,8.0,1.0,3.0,2.0,10.0,3.0,7.0,13.0,7.0,10.0,6.0,11.0,7.0,13.0,12.0,15.0,18.0,23.0,30.0,31.0,23.0,60.0,33.0,76.0,74.0,96.0,136.0,155.0,217.0,267.0,326.0,438.0,539.0,723.0,881.0,1080.0,1213.0,1466.0,1908.0,2339.0,2881.0,3447.0,4166.0,4839.0,5358.0,5564.0,5742.0,5309.0,4719.0,4145.0,3632.0,3316.0,2987.0,2670.0,2535.0,2319.0,2361.0,2078.0,1818.0,1715.0,1532.0,1858.0,1544.0,1567.0,1466.0,1260.0,1262.0,901.0,765.0,637.0,333.0,242.0
Собчак Ксения Анатольевна,27919.0,36272.0,18530.0,7780.0,3538.0,1593.0,733.0,432.0,212.0,109.0,54.0,44.0,14.0,13.0,12.0,10.0,5.0,5.0,7.0,1.0,0.0,1.0,2.0,1.0,1.0,3.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Сурайкин Максим Александрович,43232.0,48685.0,3955.0,766.0,253.0,130.0,82.0,42.0,37.0,30.0,54.0,18.0,5.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Титов Борис Юрьевич,49792.0,39856.0,6509.0,824.0,174.0,49.0,23.0,22.0,17.0,3.0,3.0,5.0,1.0,1.0,4.0,4.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Явлинский Григорий Алексеевич,51220.0,30704.0,9033.0,3637.0,1546.0,676.0,282.0,110.0,43.0,11.0,16.0,5.0,1.0,0.0,5.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0